## Summarization

In [1]:
# ! pip install datasets evaluate transformers rouge-score nltk

In [2]:
# model_checkpoint = "t5-small"
model_checkpoint = "EleutherAI/pythia-160m"
# model_checkpoint = "RWKV/rwkv-4-169m-pile"

In [3]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("xsum")
metric = load("rouge")
raw_datasets

2024-03-28 17:20:47.749987: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-28 17:20:47.751334: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-28 17:20:47.770406: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 17:20:47.770422: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 17:20:47.770436: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [4]:
raw_datasets["train"][0]

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [5]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(raw_datasets["train"])

,document,summary,id
0,"And it will be a particularly poignant day for Winston Churchill's ""secret army"", who will be parading at the Cenotaph on Remembrance Sunday for the first time.\nIt was 1940, much of Europe had fallen to the Nazis and an invasion of Britain seemed imminent.\nChurchill called it the nation's ""darkest hour"" and for one Cornish teenager it was the beginning of a great adventure.\nTrevor Miners, 86, from Perranporth, was part of Churchill's Auxiliary Units, a volunteer force trained to be the underground line of defence in the event of a German invasion.\nMr Miners was 16-years-old when, in 1943, he was asked to sign the Official Secrets Act and found himself one of 3,500 volunteers recruited to Britain's resistance army.\n""We were sent to a base in Oxfordshire,"" Mr Miners said.\n""We were trained to kill, how to use a knife to kill a man quietly.\n""The plan was that when the invasion came our unit would hide in an underground bunker and let the Nazis roll over the top of us.\n""Then after a month we were to come out at night and attack them, destroy their munitions dumps, railway lines, things like that.""\nMr Miners' bunker at Cligga Head near Perranporth was one of about 500 such bunkers, known as Operational Bases, which were dug all across Britain.\n""It wasn't as concealed as it should have been,"" said Mr Miners. ""An RAF corporal thought it would be the ideal courting spot, which was a bit embarrassing for us.\n""We had to move to another base at a former tin mine nearby.""\nThe individual units were entirely self-contained, with no knowledge of the identity or locations of the other resistance groups, although there are thought to have been about 250 volunteers in Cornwall.\n""We were only a small unit of seven men,"" he said. ""After Dunkirk it looked as if the Germans would invade right away.\n""We had food rations for five weeks in the bunker and we had any amount of explosives.\n""In fact, we were so well equipped it was unbelievable.""\n""I'm sure they would have found us in the end. So fortunately for us the invasion never happened.""\nThe bunker was about 15ft (4.6m) below ground, but has been filled in since the war.\nHistorian Tom Sykes said: ""These men were signing up to a suicide mission.\n""There was no way out for them, they were going to be caught and tortured, they were ready to kill themselves before allowing themselves to be captured.""\nThe Auxiliary Units were stood down in 1944 when the threat of invasion was finally lifted. Churchill's secret army became Churchill's forgotten army.\n""We had signed the Official Secrets Act, you see,"" Mr Miners said.\n""We would never talk about what we were trained to do. One of my unit was even sent a white feather by someone who thought he was a coward for not going out to fight, but we knew different.""\nMr Sykes has campaigned for years to have the work of the Auxiliary Units officially acknowledged.\nNow they have been granted permission to take part in the Remembrance Day parade at the Cenotaph on Sunday\nIt is unclear how many of the men who would have made up Britain's resistance army remain alive, but Mr Miners will be making the journey from Perranporth.\n""I wish my friends could be there with me,"" he said.\n""I wish this could have been done years ago so they could have been there with me but I will march to represent all of them.""","This Sunday, people across Britain will remember those who sacrificed their lives for the country in World War Two.",24851203
1,"Figures from the Society for Cardiothoracic Surgery showed the unit at University Hospital of Wales (UHW) had a survival rate of 98.67%.\nIt was placed with Southampton and Papworth as the three best-performing.\nThe figures, from April 2011 to March 2014, suggest Morriston Hospital, Swansea, was ranked sixth out of 39 units.\nIts cardiac centre started work on a Â£6.6m upgrade earlier this month.\nUHW's lead cardiac surgeon Indu Deglurkar said the NHS in Wales stil

In [7]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [8]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [9]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # if tokenizer.eos_token is not None:
    #     tokenizer.pad_token = tokenizer.eos_token
    # else:
    #     # If the tokenizer does not have an EOS token, you need to add a new pad token
    #     # This is an example, you may need to adjust it depending on the tokenizer
    #     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
if model_checkpoint in ["t5-small", "EleutherAI/pythia-160m", "RWKV/rwkv-4-169m-pile"]:
    prefix = "summarize: "
else:
    prefix = ""

In [11]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[2204, 4175, 907, 27, 380, 2120, 2105, 273, 4723, 275, 19608, 19498, 13, 581, 273, 253, 3672, 9065, 5876, 13, 310, 1335, 1146, 7515, 15, 187, 6802, 1094, 789, 310, 10800, 275, 9221, 781, 285, 1142, 13939, 275, 367, 1796, 67, 1417, 11094, 3464, 16426, 5876, 407, 6306, 1824, 15, 187, 14463, 968, 327, 253, 8935, 8852, 2022, 1282, 2454, 20638, 1955, 281, 4723, 387, 253, 418, 6472, 1299, 17721, 324, 10475, 15, 187, 10195, 9341, 285, 11692, 398, 497, 5876, 407, 28037, 275, 19608, 19498, 846, 253, 7121, 330, 658, 19068, 264, 715, 253, 3874, 15, 187, 6407, 8308, 12951, 6836, 43388, 32719, 11580, 253, 2170, 281, 16030, 253, 4723, 15, 187, 510, 12685, 35369, 247, 26179, 3402, 13, 28037, 1142, 6264, 3607, 327, 16225, 5720, 428, 253, 2022, 12701, 11080, 17937, 15, 187, 36751, 5464, 42486, 13, 665, 21186, 253, 330, 27992, 45013, 534, 369, 16426, 5876, 13, 753, 703, 812, 417, 9331, 253, 4471, 14, 47746, 2380, 2378, 253, 10802, 4352, 15, 187, 6436, 13, 703, 753, 625, 3657, 800, 789, 81

In [12]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [13]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

comet_ml is installed but `COMET_API_KEY` is not set.


In [14]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [16]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [17]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()

wandb: Currently logged in as: yimei-yang. Use `wandb login --relogin` to force relogin


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.54 GiB. GPU 0 has a total capacty of 23.63 GiB of which 396.38 MiB is free. Including non-PyTorch memory, this process has 22.58 GiB memory in use. Of the allocated memory 21.97 GiB is allocated by PyTorch, and 157.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF